In [47]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [48]:
df = pd.read_csv("/notebook/sample_text.csv")
df.shape

(8, 2)

In [49]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [50]:
from langchain.embeddings import HuggingFaceEmbeddings

# Use the updated HuggingFaceEmbeddings with the specified model
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
vectors = [embeddings.embed_query(text) for text in df.text]

In [51]:
import numpy as np
vectors_array = np.array(vectors)
print(vectors_array.shape)

(8, 768)


In [52]:
dim = vectors_array.shape[1]
dim

768

In [53]:
import faiss

index = faiss.IndexFlatL2(dim)

In [54]:
index.add(vectors_array)

In [55]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x182fcaee0> >

In [56]:
search_query = "An apple a day keeps the doctor away"
query_vector = embeddings.embed_query(search_query)

In [57]:
query_vector_array = np.array(query_vector)
query_vector_array.shape

(768,)

In [58]:
query_vector_array

array([-5.90281114e-02,  2.27091499e-02, -1.82097591e-02,  8.13040388e-05,
        4.18209508e-02,  2.65850103e-03,  6.76208287e-02,  2.03610342e-02,
       -3.30464169e-02, -5.26952669e-02, -2.18400601e-02,  3.29930671e-02,
       -4.20769043e-02,  2.53535919e-02,  1.03528807e-02,  2.86303572e-02,
        1.81760397e-02,  2.05438361e-02,  1.70531799e-03, -1.08795520e-03,
        6.38532918e-03, -4.51361714e-03,  1.83859151e-02,  4.36109751e-02,
        1.97592694e-02, -6.92133419e-03, -1.96340885e-02,  1.87755395e-02,
       -3.29025164e-02,  2.80651320e-02,  5.54695092e-02,  1.77613273e-03,
       -4.53483546e-03,  7.34572345e-03,  4.27598953e-02,  7.18150521e-03,
        5.57003170e-03, -4.27524410e-02,  1.61890835e-02,  5.39944274e-03,
       -3.62574607e-02,  3.94917186e-03, -1.27759902e-02, -1.17918570e-02,
       -1.88347641e-02, -1.10309729e-02, -4.85913269e-02,  3.45771387e-02,
       -3.26788761e-02, -2.24697422e-02, -6.65098801e-02, -2.65475176e-03,
        6.31931946e-02, -

In [59]:
svec = np.array(query_vector).reshape(1, -1)
svec.shape

(1, 768)

In [60]:
svec

array([[-5.90281114e-02,  2.27091499e-02, -1.82097591e-02,
         8.13040388e-05,  4.18209508e-02,  2.65850103e-03,
         6.76208287e-02,  2.03610342e-02, -3.30464169e-02,
        -5.26952669e-02, -2.18400601e-02,  3.29930671e-02,
        -4.20769043e-02,  2.53535919e-02,  1.03528807e-02,
         2.86303572e-02,  1.81760397e-02,  2.05438361e-02,
         1.70531799e-03, -1.08795520e-03,  6.38532918e-03,
        -4.51361714e-03,  1.83859151e-02,  4.36109751e-02,
         1.97592694e-02, -6.92133419e-03, -1.96340885e-02,
         1.87755395e-02, -3.29025164e-02,  2.80651320e-02,
         5.54695092e-02,  1.77613273e-03, -4.53483546e-03,
         7.34572345e-03,  4.27598953e-02,  7.18150521e-03,
         5.57003170e-03, -4.27524410e-02,  1.61890835e-02,
         5.39944274e-03, -3.62574607e-02,  3.94917186e-03,
        -1.27759902e-02, -1.17918570e-02, -1.88347641e-02,
        -1.10309729e-02, -4.85913269e-02,  3.45771387e-02,
        -3.26788761e-02, -2.24697422e-02, -6.65098801e-0

In [61]:
k = 2  # Number of nearest neighbors to retrieve
distances, indices = index.search(svec, k=k)

In [62]:
distances

array([[0.87064856, 1.0940377 ]], dtype=float32)

In [63]:
indices

array([[1, 0]])

In [64]:
indices.tolist()

[[1, 0]]

In [65]:
row_indices = indices.tolist()[0]
row_indices

[1, 0]

In [66]:
df.loc[row_indices]

,text,category
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
0,Meditation and yoga can improve mental health,Health


In [67]:
results = df.loc[row_indices]
results

,text,category
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
0,Meditation and yoga can improve mental health,Health


In [68]:
search_query

'An apple a day keeps the doctor away'

In [69]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel
